In [1]:
# ───────────────────────────────────────────────────────────────────────────
# Config ­— edit here
K = 15          # models per ensemble
M = 5           # independent ensembles
N_SAMPLES = 4000
SEED = 42
# ───────────────────────────────────────────────────────────────────────────

import numpy as np, pandas as pd
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, XGBRFClassifier

def stability_rmse(mat):
    return np.sqrt(2 * mat.var(axis=0, ddof=0)).mean()

def run_one_xgb(Xtr, ytr, Xte, yte, order, **xgb_kw):
    model = XGBClassifier(n_jobs=1, eval_metric="auc", use_label_encoder=False,
                          verbosity=0, **xgb_kw)
    model.fit(Xtr[order], ytr[order])
    p = model.predict_proba(Xte)[:, 1]
    return accuracy_score(yte, p > 0.5), roc_auc_score(yte, p), p

def experiment(k=15, m=5, n=4000, seed=42):
    X, y = make_classification(n_samples=n, n_features=20, n_informative=10,
                               n_redundant=5, flip_y=0.05, class_sep=1.0,
                               random_state=seed)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25,
                                          stratify=y, random_state=seed)

    rows = []

    # ── (1) Single boosted models ─────────────────────────────────────────
    sing_preds, acc, auc = [], [], []
    for i in range(k):
        order = np.random.permutation(len(Xtr))
        a, u, p = run_one_xgb(
            Xtr, ytr, Xte, yte, order,
            n_estimators=120, learning_rate=0.1, max_depth=4,
            subsample=0.9, colsample_bytree=0.9,
            tree_method="hist", random_state=seed+i)
        acc.append(a); auc.append(u); sing_preds.append(p)
    rows.append(dict(Variant=f"Single XGB (K={k})",
                     Accuracy=np.mean(acc), ROC_AUC=np.mean(auc),
                     Stability_RMSE=stability_rmse(np.vstack(sing_preds))))

    # ── (2) Ensembles of K models, repeated M times ───────────────────────
    ens_preds = []
    for e in range(m):
        members = []
        for i in range(k):
            order = np.random.permutation(len(Xtr))
            _, _, p = run_one_xgb(
                Xtr, ytr, Xte, yte, order,
                n_estimators=120, learning_rate=0.1, max_depth=4,
                subsample=0.9, colsample_bytree=0.9,
                tree_method="hist", random_state=seed+10_000*e+i)
            members.append(p)
        ens_preds.append(np.mean(members, axis=0))
    rows.append(dict(Variant=f"Ensemble (K={k}) × {m}",
                     Accuracy=accuracy_score(yte, ens_preds[0] > 0.5),
                     ROC_AUC=roc_auc_score(yte, ens_preds[0]),
                     Stability_RMSE=stability_rmse(np.vstack(ens_preds))))

    # ── (3) XGB Random-Forest ─────────────────────────────────────────────
    rf_preds, acc_rf, auc_rf = [], [], []
    for i in range(k):
        order = np.random.permutation(len(Xtr))
        rf = XGBRFClassifier(n_estimators=200, learning_rate=0.5, max_depth=6,
                             subsample=0.8, colsample_bytree=0.8,
                             random_state=seed+i, n_jobs=1,
                             eval_metric="auc", use_label_encoder=False,
                             verbosity=0)
        rf.fit(Xtr[order], ytr[order])
        p = rf.predict_proba(Xte)[:, 1]
        acc_rf.append(accuracy_score(yte, p > 0.5))
        auc_rf.append(roc_auc_score(yte, p))
        rf_preds.append(p)
    rows.append(dict(Variant="XGB Random-Forest",
                     Accuracy=np.mean(acc_rf), ROC_AUC=np.mean(auc_rf),
                     Stability_RMSE=stability_rmse(np.vstack(rf_preds))))

    # ── (4) Fully deterministic boosted model ────────────────────────────
    #     tree_method='exact', subsample=1, colsample_bytree=1
    det_preds, acc_det, auc_det = [], [], []
    for i in range(k):
        order = np.random.permutation(len(Xtr))
        a, u, p = run_one_xgb(
            Xtr, ytr, Xte, yte, order,
            n_estimators=120, learning_rate=0.1, max_depth=4,
            subsample=1.0, colsample_bytree=1.0,
            tree_method="exact", random_state=seed+i)
        acc_det.append(a); auc_det.append(u); det_preds.append(p)
    rows.append(dict(Variant="XGB Exact (subsample=1, colsample=1)",
                     Accuracy=np.mean(acc_det), ROC_AUC=np.mean(auc_det),
                     Stability_RMSE=stability_rmse(np.vstack(det_preds))))

    return pd.DataFrame(rows).round(4)

# Run and display
df_res = experiment(K, M, N_SAMPLES, SEED)
df_res

,Variant,Accuracy,ROC_AUC,Stability_RMSE
0,Single XGB (K=15),0.9285,0.9643,0.0313
1,Ensemble (K=15) × 5,0.9310,0.9647,0.0072
2,XGB Random-Forest,0.8957,0.9514,0.0086
3,"XGB Exact (subsample=1, colsample=1)",0.9250,0.9632,0.0000


## Proof that histogram binning is behind the issue

In [2]:
import numpy as np, pandas as pd
from xgboost import XGBClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

def stability_rmse(mat):
    return np.sqrt(2 * mat.var(axis=0, ddof=0)).mean()

def run_variant(n_rows, n_jobs, tree_method, subsample, n_shuffles=10, seed=0):
    # fresh data set for each size
    X, y = make_classification(
        n_samples=n_rows, n_features=30, n_informative=15, n_redundant=10,
        class_sep=1.0, random_state=seed)
    Xtr, Xte, ytr, yte = train_test_split(
        X, y, test_size=0.25, stratify=y, random_state=seed)

    preds = []
    for s in range(n_shuffles):
        order = np.random.RandomState(seed + s).permutation(len(Xtr))
        mdl = XGBClassifier(
            n_estimators=120, max_depth=5, learning_rate=0.1,
            subsample=subsample, colsample_bytree=1.0,
            tree_method=tree_method, n_jobs=n_jobs,
            random_state=42, eval_metric="auc",
            use_label_encoder=False, verbosity=0)
        mdl.fit(Xtr[order], ytr[order])
        preds.append(mdl.predict_proba(Xte)[:, 1])
    return stability_rmse(np.vstack(preds))

# ------------------------------------------------------------------
variants = [
    # large-data, multithread
    ("large-hist-1.0",   dict(n_rows=60_000, n_jobs=8, tree_method="hist",  subsample=1.0)),
    ("large-hist-0.8",   dict(n_rows=60_000, n_jobs=8, tree_method="hist",  subsample=0.8)),
    ("large-exact",      dict(n_rows=60_000, n_jobs=8, tree_method="exact", subsample=1.0)),
    # small-data, single-thread
    ("small-hist-1.0",   dict(n_rows=6_000,  n_jobs=1, tree_method="hist",  subsample=1.0)),
    ("small-hist-0.8",   dict(n_rows=6_000,  n_jobs=1, tree_method="hist",  subsample=0.8)),
    ("small-exact",      dict(n_rows=6_000,  n_jobs=1, tree_method="exact", subsample=1.0)),
]

results = []
for name, cfg in variants:
    rmse = run_variant(**cfg)
    results.append({"variant": name, "rmse": round(rmse, 4)})

pd.DataFrame(results)

,variant,rmse
0,large-hist-1.0,0.0293
1,large-hist-0.8,0.0326
2,large-exact,0.0000
3,small-hist-1.0,0.0000
4,small-hist-0.8,0.0388
5,small-exact,0.0000


## Other Supervised Algoritms

In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Import all models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
import lightgbm as lgb

# Optional imports (install if testing)
try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
except ImportError:
    CATBOOST_AVAILABLE = False

def test_algorithm_stability(X_train, y_train, X_test, model_class, model_params=None, n_shuffles=10):
    """Test if an algorithm gives consistent predictions under row shuffling"""
    if model_params is None:
        model_params = {}
    
    predictions = []
    
    for shuffle_idx in range(n_shuffles):
        # Create a shuffled version of the training data
        shuffle_indices = np.random.RandomState(shuffle_idx).permutation(len(X_train))
        X_shuffled = X_train[shuffle_indices]
        y_shuffled = y_train[shuffle_indices]
        
        # Train model
        model = model_class(**model_params)
        
        # Special handling for different model types
        if 'random_state' in model.get_params().keys() and 'random_state' not in model_params:
            model.set_params(random_state=42)
        
        model.fit(X_shuffled, y_shuffled)
        
        # Get predictions
        preds = model.predict(X_test)
        predictions.append(preds)
    
    # Calculate stability metrics
    predictions = np.array(predictions)
    
    # Per-sample standard deviation
    std_per_sample = np.std(predictions, axis=0)
    
    # Average pairwise RMSE
    pairwise_rmses = []
    for i in range(n_shuffles):
        for j in range(i+1, n_shuffles):
            rmse = np.sqrt(np.mean((predictions[i] - predictions[j])**2))
            pairwise_rmses.append(rmse)
    
    return {
        'mean_std': np.mean(std_per_sample),
        'max_std': np.max(std_per_sample),
        'mean_pairwise_rmse': np.mean(pairwise_rmses),
        'max_pairwise_rmse': np.max(pairwise_rmses),
        'is_stable': np.max(std_per_sample) < 1e-10  # Numerical precision threshold
    }

def investigate_random_forest():
    """Deep dive into why Random Forest shows such high variance"""
    print("\n\nINVESTIGATING RANDOM FOREST INSTABILITY")
    print("=" * 80)
    
    X, y = make_regression(n_samples=500, n_features=10, noise=0.1, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Test different RF configurations
    rf_configs = [
        ("RF (default)", {}),
        ("RF (bootstrap=False)", {'bootstrap': False}),
        ("RF (max_samples=1.0)", {'max_samples': 1.0}),
        ("RF (n_estimators=1)", {'n_estimators': 1}),  # Single tree
        ("RF (warm_start=True)", {'warm_start': True}),
    ]
    
    print(f"{'Configuration':<30} {'Mean RMSE':<15} {'Notes'}")
    print("-" * 80)
    
    for name, extra_params in rf_configs:
        params = {'n_estimators': 10, 'max_depth': 5, 'random_state': 42}
        params.update(extra_params)
        
        try:
            result = test_algorithm_stability(X_train, y_train, X_test, RandomForestRegressor, params, n_shuffles=5)
            print(f"{name:<30} {result['mean_pairwise_rmse']:<15.6f}")
            
            # Let's also check what's happening with predictions
            if name == "RF (default)":
                # Get two predictions with different orderings
                model1 = RandomForestRegressor(**params)
                model1.fit(X_train, y_train)
                pred1 = model1.predict(X_test[:5])
                
                idx = np.random.permutation(len(X_train))
                model2 = RandomForestRegressor(**params)
                model2.fit(X_train[idx], y_train[idx])
                pred2 = model2.predict(X_test[:5])
                
                print(f"\n  Sample predictions comparison (first 5):")
                print(f"  Original order: {pred1}")
                print(f"  Shuffled order: {pred2}")
                print(f"  Difference:     {pred1 - pred2}")
                
        except Exception as e:
            print(f"{name:<30} ERROR: {str(e)}")

def run_comprehensive_test():
    """Test multiple algorithms for row-order stability"""
    
    # Create dataset
    X, y = make_regression(n_samples=2000, n_features=20, noise=0.1, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Scale features for algorithms that need it
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Define algorithms to test
    algorithms = [
        # Should be stable
        ("Linear Regression", LinearRegression, {}, False),
        ("Ridge Regression", Ridge, {'alpha': 1.0}, False),
        ("Decision Tree (no sampling)", DecisionTreeRegressor, {'max_depth': 5, 'random_state': 42}, False),
        
        # Potentially unstable - coordinate descent
        ("Lasso (coord descent)", Lasso, {'alpha': 0.1, 'random_state': 42, 'selection': 'cyclic'}, False),
        ("Lasso (random coord descent)", Lasso, {'alpha': 0.1, 'random_state': 42, 'selection': 'random'}, False),
        ("ElasticNet", ElasticNet, {'alpha': 0.1, 'random_state': 42}, False),
        
        # Potentially unstable - SGD
        ("SGD Regressor", SGDRegressor, {'random_state': 42, 'max_iter': 1000}, True),
        ("SGD (different loss)", SGDRegressor, {'loss': 'huber', 'random_state': 42, 'max_iter': 1000}, True),
        ("MLP (adam)", MLPRegressor, {'hidden_layer_sizes': (50,), 'random_state': 42, 'max_iter': 500}, True),
        ("MLP (sgd)", MLPRegressor, {'hidden_layer_sizes': (50,), 'solver': 'sgd', 'random_state': 42, 'max_iter': 500}, True),
        ("MLP (lbfgs)", MLPRegressor, {'hidden_layer_sizes': (50,), 'solver': 'lbfgs', 'random_state': 42, 'max_iter': 500}, True),
        
        # Tree-based methods
        ("Random Forest", RandomForestRegressor, {'n_estimators': 50, 'max_depth': 5, 'random_state': 42}, False),
        ("XGBoost (hist)", xgb.XGBRegressor, {'n_estimators': 50, 'max_depth': 5, 'tree_method': 'hist', 'random_state': 42}, False),
        ("XGBoost (exact)", xgb.XGBRegressor, {'n_estimators': 50, 'max_depth': 5, 'tree_method': 'exact', 'random_state': 42}, False),
        ("XGBoost (exact, no subsample)", xgb.XGBRegressor, {'n_estimators': 50, 'max_depth': 5, 'tree_method': 'exact', 'subsample': 1.0, 'colsample_bytree': 1.0, 'random_state': 42}, False),
        ("LightGBM", lgb.LGBMRegressor, {'n_estimators': 50, 'max_depth': 5, 'random_state': 42, 'verbose': -1}, False),
        ("LightGBM (deterministic)", lgb.LGBMRegressor, {'n_estimators': 50, 'max_depth': 5, 'deterministic': True, 'force_row_wise': True, 'random_state': 42, 'verbose': -1}, False),
        
        # Other algorithms
        ("SVM (RBF)", SVR, {'kernel': 'rbf', 'gamma': 'scale'}, True),  # No random_state param
        ("KNN", KNeighborsRegressor, {'n_neighbors': 5}, False),
    ]
    
    # Add CatBoost if available
    if CATBOOST_AVAILABLE:
        algorithms.append(("CatBoost", cb.CatBoostRegressor, {'iterations': 50, 'depth': 5, 'random_state': 42, 'verbose': False}, False))
    
    # Test each algorithm
    results = []
    
    print("Testing algorithm stability under row permutation...")
    print("=" * 80)
    print(f"{'Algorithm':<35} {'Mean STD':<12} {'Max STD':<12} {'Mean RMSE':<12} {'Stable?':<10}")
    print("-" * 80)
    
    for name, model_class, params, use_scaled in algorithms:
        X_tr = X_train_scaled if use_scaled else X_train
        X_te = X_test_scaled if use_scaled else X_test
        
        try:
            result = test_algorithm_stability(X_tr, y_train, X_te, model_class, params)
            
            results.append({
                'algorithm': name,
                **result
            })
            
            stability = "YES" if result['is_stable'] else "NO"
            print(f"{name:<35} {result['mean_std']:<12.2e} {result['max_std']:<12.2e} {result['mean_pairwise_rmse']:<12.2e} {stability:<10}")
            
        except Exception as e:
            print(f"{name:<35} ERROR: {str(e)[:70]}")
    
    print("\n" + "=" * 80)
    print("\nSUMMARY:")
    print("-" * 40)
    
    # Categorize results
    stable_algos = [r['algorithm'] for r in results if r['is_stable']]
    unstable_algos = [r['algorithm'] for r in results if not r['is_stable']]
    
    print(f"\nSTABLE ALGORITHMS ({len(stable_algos)}):")
    for algo in stable_algos:
        print(f"  ✓ {algo}")
    
    print(f"\nUNSTABLE ALGORITHMS ({len(unstable_algos)}):")
    for algo in unstable_algos:
        result = next(r for r in results if r['algorithm'] == algo)
        print(f"  ✗ {algo} (avg RMSE between runs: {result['mean_pairwise_rmse']:.4f})")
    
    return results

def test_specific_cases():
    """Test specific interesting cases in more detail"""
    
    print("\n\nDETAILED ANALYSIS OF SPECIFIC CASES")
    print("=" * 80)
    
    X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Case 1: XGBoost with different settings
    print("\n1. XGBoost with different configurations:")
    print("-" * 40)
    
    xgb_configs = [
        ("XGB (hist, subsample=0.8)", {'tree_method': 'hist', 'subsample': 0.8}),
        ("XGB (hist, subsample=1.0)", {'tree_method': 'hist', 'subsample': 1.0}),
        ("XGB (exact, subsample=0.8)", {'tree_method': 'exact', 'subsample': 0.8}),
        ("XGB (exact, subsample=1.0)", {'tree_method': 'exact', 'subsample': 1.0}),
        ("XGB (exact, all 1.0)", {'tree_method': 'exact', 'subsample': 1.0, 'colsample_bytree': 1.0}),
    ]
    
    for name, extra_params in xgb_configs:
        params = {'n_estimators': 50, 'max_depth': 5, 'random_state': 42}
        params.update(extra_params)
        
        result = test_algorithm_stability(X_train, y_train, X_test, xgb.XGBRegressor, params, n_shuffles=5)
        stable = "YES" if result['is_stable'] else "NO"
        print(f"{name:<35} RMSE: {result['mean_pairwise_rmse']:.6f}, Stable: {stable}")
    
    # Case 2: Neural networks with different batch sizes
    print("\n2. Neural Networks with different batch sizes:")
    print("-" * 40)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    mlp_configs = [
        ("MLP (batch_size=1000)", {'batch_size': 1000}),  # Full batch
        ("MLP (batch_size=200)", {'batch_size': 200}),    # Large batch
        ("MLP (batch_size=32)", {'batch_size': 32}),      # Mini-batch
        ("MLP (batch_size=1)", {'batch_size': 1}),        # SGD
    ]
    
    for name, extra_params in mlp_configs:
        params = {'hidden_layer_sizes': (50,), 'random_state': 42, 'max_iter': 200, 'learning_rate_init': 0.01}
        params.update(extra_params)
        
        result = test_algorithm_stability(X_train_scaled, y_train, X_test_scaled, MLPRegressor, params, n_shuffles=5)
        stable = "YES" if result['is_stable'] else "NO"
        print(f"{name:<35} RMSE: {result['mean_pairwise_rmse']:.6f}, Stable: {stable}")

if __name__ == "__main__":
    # Run comprehensive test
    results = run_comprehensive_test()
    
    # Investigate Random Forest
    investigate_random_forest()
    
    # Run detailed analysis
    test_specific_cases()
    
    print("\n\nKEY INSIGHTS:")
    print("=" * 80)
    print("""
Based on the empirical results:

1. **Perfectly Stable** (deterministic):
   - Linear/Ridge regression (closed-form solutions)
   - Decision Trees with fixed random_state
   - Coordinate descent methods (even with 'random' selection!)
   - KNN (no randomness involved)
   - LightGBM (surprisingly stable by default)

2. **Unstable Algorithms**:
   - SGD-based methods (mini-batch order matters)
   - Neural networks (batch formation depends on order)
   - Random Forest (high variance - needs investigation)
   - XGBoost (small but measurable differences)

3. **Key Findings**:
   - sklearn's coordinate descent properly controls randomness
   - LightGBM appears more stable than XGBoost by default
   - Random Forest's instability is surprisingly large
   - Even 'exact' tree methods can be unstable with subsampling

4. **To Achieve Stability**:
   - Use algorithms with closed-form solutions when possible
   - For XGBoost: tree_method='exact', subsample=1.0, colsample_bytree=1.0
   - For neural networks: use full-batch training (lbfgs solver)
   - For ensemble methods: be aware of the variance/stability trade-off
    """)

Testing algorithm stability under row permutation...
Algorithm                           Mean STD     Max STD      Mean RMSE    Stable?   
--------------------------------------------------------------------------------
Linear Regression                   1.99e-13     5.43e-13     3.03e-13     YES       
Ridge Regression                    8.94e-14     2.30e-13     1.36e-13     YES       
Decision Tree (no sampling)         1.39e-14     5.68e-14     0.00e+00     YES       
Lasso (coord descent)               3.63e-14     1.19e-13     5.01e-14     YES       
Lasso (random coord descent)        3.75e-14     1.25e-13     5.19e-14     YES       
ElasticNet                          3.94e-14     1.21e-13     5.50e-14     YES       
SGD Regressor                       2.90e-03     6.01e-03     4.42e-03     NO        
SGD (different loss)                7.79e-03     1.73e-02     1.19e-02     NO        
MLP (adam)                          3.63e-01     1.27e+00     6.14e-01     NO        
MLP (s

### Other Unsupervised Algorithms

In [4]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import adjusted_rand_score

# Create dataset
X, _ = make_blobs(n_samples=1000, n_features=5, centers=4, cluster_std=1.0, random_state=123)

# General function
def compute_mean_ari(X, cluster_func, n_shuffles=10, seed=42):
    labels_list = []
    for i in range(n_shuffles):
        rng = np.random.RandomState(seed + i)
        order = rng.permutation(len(X))
        X_shuffled = X[order]
        labels = cluster_func(X_shuffled)
        inverse = np.argsort(order)
        labels_unshuffled = labels[inverse]
        labels_list.append(labels_unshuffled)

    aris = [adjusted_rand_score(labels_list[i], labels_list[j])
            for i in range(n_shuffles) for j in range(i + 1, n_shuffles)]
    return np.mean(aris)

# Define methods
methods = {
    "KMeans": lambda X: KMeans(n_clusters=4, random_state=42).fit(X).labels_,
    "Agglomerative": lambda X: AgglomerativeClustering(n_clusters=4).fit(X).labels_,
    "GMM": lambda X: GaussianMixture(n_components=4, random_state=42).fit(X).predict(X),
    "DBSCAN": lambda X: DBSCAN(eps=1.2).fit(X).labels_,
}

# Run tests
results = []
for name, func in methods.items():
    try:
        mean_ari = compute_mean_ari(X, func)
        results.append({"algorithm": name, "mean_ari": round(mean_ari, 4)})
    except Exception as e:
        results.append({"algorithm": name, "mean_ari": None, "error": str(e)})

# Display
pd.DataFrame(results)

,algorithm,mean_ari
0,KMeans,0.8672
1,Agglomerative,1.0000
2,GMM,0.8677
3,DBSCAN,1.0000


In [5]:
import numpy as np, pandas as pd
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.metrics import adjusted_rand_score

# ------------------------------------------------------------------
# Helper: mean ARI across shuffled fits
def mean_ari_row_shuffles(X, cluster_func, n_shuffles=10, seed=42):
    assignments = []
    for i in range(n_shuffles):
        order = np.random.RandomState(seed + i).permutation(len(X))
        labels = cluster_func(X[order])
        # restore original row order
        aligned = np.empty_like(labels)
        aligned[np.argsort(order)] = labels
        assignments.append(aligned)

    return np.mean([
        adjusted_rand_score(assignments[i], assignments[j])
        for i in range(n_shuffles) for j in range(i + 1, n_shuffles)
    ])

# ------------------------------------------------------------------
# Datasets designed to stress DBSCAN
datasets = {
    "blobs_hi_noise": make_blobs(n_samples=1000, centers=4, cluster_std=3.5,
                                 random_state=123)[0],
    "blobs_rounded":  np.round(
        make_blobs(n_samples=1000, centers=4, cluster_std=2.5,
                   random_state=123)[0], 1),
    "moons_noisy":    StandardScaler().fit_transform(
        make_moons(n_samples=1000, noise=0.25, random_state=123)[0]),
    "circles_noisy":  StandardScaler().fit_transform(
        make_circles(n_samples=1000, factor=0.3, noise=0.18,
                     random_state=123)[0]),
}

# ------------------------------------------------------------------
# Parameter grid to explore
param_grid = [
    {"eps": 1.2, "min_samples": 5},
    {"eps": 0.9, "min_samples": 5},
    {"eps": 0.9, "min_samples": 3},
    {"eps": 0.7, "min_samples": 3},
    {"eps": 0.6, "min_samples": 2},
]

# ------------------------------------------------------------------
rows = []
for dname, X in datasets.items():
    for params in param_grid:
        eps, ms = params["eps"], params["min_samples"]

        def run_dbscan(Z):  # closure captures eps & ms
            return DBSCAN(eps=eps, min_samples=ms).fit(Z).labels_

        ari = mean_ari_row_shuffles(X, run_dbscan)
        rows.append({
            "dataset": dname,
            "eps": eps,
            "min_samples": ms,
            "mean_ari": round(ari, 4),
        })

results = pd.DataFrame(rows).pivot_table(
    index=["dataset"], columns=["eps", "min_samples"], values="mean_ari"
)
results

eps                0.6     0.7     0.9            1.2
min_samples          2       3       3      5       5
dataset                                              
blobs_hi_noise -0.0048  0.0017  0.0017  0.001  0.0074
blobs_rounded  -0.0010 -0.0041  0.0050 -0.002  0.0034
circles_noisy  -0.0010  1.0000  1.0000  1.000  1.0000
moons_noisy    -0.0010 -0.0010  1.0000  1.000  1.0000